<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/NM/%D0%9B%D0%B5%D0%BC%D0%B0%D1%82%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F_%D0%92%D0%B5%D0%BA%D1%82%D0%BE%D1%80%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install pymorphy2 langdetect natasha

  Using cached pymorphy2-0.9.1-py3-none-any.whl.metadata (3.6 kB)
  Using cached langdetect-1.0.9.tar.gz (981 kB)
  Preparing metadata (setup.py) ... done
  Using cached natasha-1.6.0-py3-none-any.whl.metadata (23 kB)
  Using cached DAWG_Python-0.7.2-py2.py3-none-any.whl.metadata (7.0 kB)
  Using cached pymorphy2_dicts_ru-2.4.417127.4579844-py2.py3-none-any.whl.metadata (2.1 kB)
  Using cached docopt-0.6.2.tar.gz (25 kB)
  Preparing metadata (setup.py) ... done
  Using cached razdel-0.5.0-py3-none-any.whl.metadata (10.0 kB)
  Using cached navec-0.10.0-py3-none-any.whl.metadata (21 kB)
  Using cached slovnet-0.6.0-py3-none-any.whl.metadata (34 kB)
  Using cached yargy-0.16.0-py3-none-any.whl.metadata (3.5 kB)
  Using cached ipymarkup-0.9.0-py3-none-any.whl.metadata (5.6 kB)
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━

In [29]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize, regexp_tokenize
from pymorphy2 import MorphAnalyzer
from transformers import AutoTokenizer
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from langdetect import detect, LangDetectException
import spacy
import json
import os
from natasha import MorphVocab
from wordcloud import WordCloud
import subprocess
import sys

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt_tab')

class TextProcessor:
    def __init__(self, text, language=None):
        self.text = text
        self.language = language if language else self.detect_language()
        self.morph_vocab = MorphVocab() if self.language == 'ru' else None
        self.morph_analyzer = MorphAnalyzer() if self.language == 'ru' else None
        self.stop_words = set(stopwords.words(self.language)) if self.language in stopwords.fileids() else set()
        self.nlp = self._load_spacy_model() if self.language in {'en', 'de'} else None
        self.results = []  # Для хранения результатов методов

    def _load_spacy_model(self):
        """
        Загружает модель spaCy для указанного языка.
        Если модель не установлена, пытается автоматически её установить.
        """
        model_map = {
            'en': "en_core_web_sm",
            'de': "de_core_news_sm"
        }

        if self.language in model_map:
            model_name = model_map[self.language]

            # Проверяем, установлена ли модель
            if not spacy.util.is_package(model_name):
                print(f"spaCy model '{model_name}' is not installed. Attempting to install...")

                # Команда для установки модели
                try:
                    subprocess.run([sys.executable, "-m", "spacy", "download", model_name], check=True)
                    print(f"Model '{model_name}' has been successfully installed.")
                except subprocess.CalledProcessError as e:
                    raise RuntimeError(f"Failed to install spaCy model '{model_name}'. Error: {e}")

            # Пытаемся загрузить модель
            try:
                return spacy.load(model_name)
            except OSError:
                raise NotImplementedError(f"spaCy model for language '{self.language}' is not available after installation attempt.")

        else:
            print(f"spaCy does not support the language '{self.language}'. Using alternative NLP tools.")
            return None

    def clean_text(self):
        cleaned_text = re.sub(r"http\S+", "", self.text)
        cleaned_text = re.sub(r"<.*?>", "", cleaned_text)
        cleaned_text = re.sub(r"[^a-zA-Zа-яА-ЯёЁ0-9\s]", "", cleaned_text)
        return cleaned_text.strip()

    def remove_stopwords(self, tokens=None):
        if tokens is None:
            tokens = self.standard_tokenization()
        return [token for token in tokens if token.lower() not in self.stop_words]

    def simple_tokenization(self):
        tokens = self.clean_text().split()
        self._save_result("Simple Tokenization", tokens)
        return tokens

    def standard_tokenization(self):
        tokens = word_tokenize(self.clean_text())
        self._save_result("Standard Tokenization", tokens)
        return tokens

    def character_tokenization(self):
        tokens = list(self.clean_text())
        self._save_result("Character Tokenization", tokens)
        return tokens

    def byte_level_tokenization(self):
        tokens = [byte for byte in self.clean_text().encode('utf-8')]
        self._save_result("Byte-Level Tokenization", tokens)
        return tokens

    def zipf_law_analysis(self, save_path="zipf_law.png"):
        """
        Выполняет анализ закона Ципфа и сохраняет график в файл.
        """
        tokens = self.remove_stopwords(self.standard_tokenization())
        freq_dist = Counter(tokens)
        frequencies = sorted(freq_dist.values(), reverse=True)
        ranks = range(1, len(frequencies) + 1)

        plt.figure(figsize=(10, 6))
        plt.plot(ranks, frequencies, marker='o')
        plt.xscale('log')
        plt.yscale('log')
        plt.xlabel('Rank')
        plt.ylabel('Frequency')
        plt.title("Zipf's Law Analysis")

        # Сохраняем график в файл
        plt.savefig(save_path)
        plt.close()

        # Сохраняем результат
        self._save_result("Zipf's Law Analysis", {"image": save_path})
        return save_path

    def visualize_word_frequencies(self, save_path="word_frequencies.png"):
        """
        Визуализирует частоты слов в виде облака слов и сохраняет изображение в файл.
        """
        tokens = self.remove_stopwords(self.standard_tokenization())
        freq_dist = Counter(tokens)
        wordcloud = WordCloud(width=800, height=400).generate_from_frequencies(freq_dist)

        plt.figure(figsize=(10, 5))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis("off")
        plt.title("Word Frequencies Visualization")

        # Сохраняем график в файл
        plt.savefig(save_path)
        plt.close()

        # Сохраняем результат
        self._save_result("Word Frequencies Visualization", {"image": save_path})
        return save_path

    def rule_based_tokenization(self, pattern=r'\w+'):
        tokens = regexp_tokenize(self.clean_text(), pattern)
        self._save_result("Rule-Based Tokenization", tokens)
        return tokens

    def stemming(self):
        """
        Выполняет стемминг текста с использованием SnowballStemmer.
        Для русского языка используется лемматизация через pymorphy2,
        так как стемминг для русского языка менее эффективен.
        """
        if self.language == 'ru':
            # Для русского языка используем лемматизацию вместо стемминга
            print("Stemming is not recommended for Russian. Using lemmatization instead.")
            tokens = self.remove_stopwords(self.standard_tokenization())
            lemmas = [self.morph_analyzer.parse(token)[0].normal_form for token in tokens]
            self._save_result("Lemmatization", lemmas)
            return lemmas

        elif self.language in SnowballStemmer.languages:
            # Для остальных языков, поддерживаемых SnowballStemmer, выполняем стемминг
            stemmer = SnowballStemmer(self.language)
            tokens = self.remove_stopwords(self.standard_tokenization())
            stemmed_tokens = [stemmer.stem(token) for token in tokens]
            self._save_result("Stemming", stemmed_tokens)
            return stemmed_tokens

        #else:
            # Если язык не поддерживается, выбрасываем исключение
            #raise ValueError(f"Stemming is not supported for language '{self.language}'.")

    def lemmatize(self, cleaned_text):
        if self.language == 'ru':
            words = word_tokenize(cleaned_text)
            lemmas = [self.morph_analyzer.parse(word)[0].normal_form for word in words]
            self._save_result("Lemmatization", lemmas)
            return lemmas
        elif self.nlp:
            doc = self.nlp(cleaned_text)
            lemmas = [token.lemma_ for token in doc]
            self._save_result("Lemmatization", lemmas)
            return lemmas
        else:
            raise NotImplementedError(f"Лемматизация для языка '{self.language}' не поддерживается.")

    def universal_lemmatization(self):
        """
        Универсальный метод лемматизации, который выбирает подходящий инструмент в зависимости от языка.
        """
        if self.language == 'ru':
            tokens = self.remove_stopwords(self.standard_tokenization())
            lemmatized_tokens = [(token, self.morph_analyzer.parse(token)[0].normal_form) for token in tokens]
        elif self.language in {'en', 'de'}:
            doc = self.nlp(self.clean_text())
            lemmatized_tokens = [(token.text, token.lemma_) for token in doc if token.text.lower() not in self.stop_words]
        else:
            raise NotImplementedError(f"Lemmatization is not implemented for language '{self.language}'.")

        self._save_result("Universal Lemmatization", lemmatized_tokens)
        return lemmatized_tokens

    def morphological_analysis(self):
        if self.language == 'ru':
            tokens = self.remove_stopwords(self.standard_tokenization())
            morph_tokens = [(token, self.morph_analyzer.parse(token)[0].normal_form) for token in tokens]
        elif self.language in {'en', 'de'}:
            doc = self.nlp(self.clean_text())
            morph_tokens = [(token.text, token.morph) for token in doc if token.text.lower() not in self.stop_words]
        else:
            raise NotImplementedError(f"Morphological analysis is not implemented for language '{self.language}'.")

        self._save_result("Morphological Analysis", morph_tokens)
        return morph_tokens

    def subword_tokenization(self, model_name="bert-base-uncased"):
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        tokens = tokenizer.tokenize(self.clean_text())
        self._save_result("Subword Tokenization", tokens)
        return tokens

    def neural_tokenization(self, model_name="bert-base-uncased"):
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        token_ids = tokenizer.encode(self.clean_text(), add_special_tokens=True)
        self._save_result("Neural Tokenization", token_ids)
        return token_ids

    def hybrid_tokenization(self):
        rule_tokens = self.rule_based_tokenization()
        if self.language in SnowballStemmer.languages:
            stemmer = SnowballStemmer(self.language)
            tokens = [stemmer.stem(token) for token in rule_tokens]
        else:
            tokens = rule_tokens
        self._save_result("Hybrid Tokenization", tokens)
        return tokens

    def language_specific_tokenization(self):
        if self.language == 'en':
            tokens = self.standard_tokenization()
        elif self.language == 'ru':
            tokens = self.morphological_analysis()
        elif self.language == 'de':
            if not self.nlp:
                raise NotImplementedError(f"Language-specific tokenization is not implemented for language '{self.language}'.")
            doc = self.nlp(self.clean_text())
            tokens = [token.text for token in doc]
        else:
            raise NotImplementedError(f"Language '{self.language}' not supported yet.")

        self._save_result("Language-Specific Tokenization", tokens)
        return tokens

    def detect_language(self):
        try:
            return detect(self.text)
        except LangDetectException:
            return "Unknown"

    def generate_ngrams(self, n=2):
        tokens = self.remove_stopwords(self.standard_tokenization())
        ngrams = zip(*[tokens[i:] for i in range(n)])
        ngrams = [" ".join(ngram) for ngram in ngrams]
        self._save_result(f"{n}-grams", ngrams)
        return ngrams

    def save_tokens_to_file(self, filename="tokens.json"):
        tokens = self.remove_stopwords(self.standard_tokenization())
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(tokens, f, ensure_ascii=False, indent=4)

    def load_tokens_from_file(self, filename="tokens.json"):
        with open(filename, "r", encoding="utf-8") as f:
            return json.load(f)

    def evaluate_tokenization(self, gold_standard_tokens):
        predicted_tokens = self.remove_stopwords(self.standard_tokenization())
        correct = len(set(predicted_tokens) & set(gold_standard_tokens))
        precision = correct / len(predicted_tokens) if predicted_tokens else 0
        recall = correct / len(gold_standard_tokens) if gold_standard_tokens else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        result = {"precision": precision, "recall": recall, "f1_score": f1_score}
        self._save_result("Tokenization Evaluation", result)
        return result

    def _save_result(self, method_name, result):
        """
        Сохраняет результат выполнения метода в список результатов.
        """
        self.results.append({
            "method": method_name,
            "text": self.text,
            "result": result
        })

    def execute_all_methods(self):
        """
        Выполняет все методы класса TextProcessor и сохраняет их результаты.
        """
        methods_to_execute = [
            ("Simple Tokenization", self.simple_tokenization),
            ("Standard Tokenization", self.standard_tokenization),
            ("Character Tokenization", self.character_tokenization),
            ("Byte-Level Tokenization", self.byte_level_tokenization),
            ("Rule-Based Tokenization", lambda: self.rule_based_tokenization(pattern=r'\w+')),
            ("Stemming", self.stemming),
            ("Lemmatization", lambda: self.lemmatize(self.clean_text())),
            ("Universal Lemmatization", self.universal_lemmatization),
            ("Morphological Analysis", self.morphological_analysis),
            ("Subword Tokenization", lambda: self.subword_tokenization(model_name="bert-base-uncased")),
            ("Neural Tokenization", lambda: self.neural_tokenization(model_name="bert-base-uncased")),
            ("Hybrid Tokenization", self.hybrid_tokenization),
            ("Language-Specific Tokenization", self.language_specific_tokenization),
            ("2-grams", lambda: self.generate_ngrams(n=2)),
            ("3-grams", lambda: self.generate_ngrams(n=3)),
            ("Zipf's Law Analysis", lambda: self.zipf_law_analysis(save_path=f"{self.language}_zipf_law.png")),
            ("Word Frequencies Visualization", lambda: self.visualize_word_frequencies(save_path=f"{self.language}_word_frequencies.png"))
        ]

        for method_name, method in methods_to_execute:
            try:
                if method_name in {"Zipf's Law Analysis", "Word Frequencies Visualization"}:
                    # Для методов, которые только рисуют графики, не сохраняем результат
                    method()
                else:
                    result = method()
                    self._save_result(method_name, result)
            except Exception as e:
                print(f"Ошибка при выполнении метода {method_name}: {e}")


    def _save_result(self, method_name, result):
            """
            Сохраняет результат выполнения метода в список результатов.
            """
            self.results.append({
                "method": method_name,
                "text": self.text,
                "result": result,
                "language": self.language  # Добавляем информацию о языке
            })

    def get_results(self):
        """
        Возвращает все сохраненные результаты.
        """
        return self.results

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [30]:
class HTMLReportGenerator:
    def __init__(self, results_list):
        """
        Инициализирует генератор HTML-отчетов.

        :param results_list: Список списков результатов из разных TextProcessor'ов.
        """
        self.results = []
        seen_results = set()  # Для отслеживания уникальности записей

        for results in results_list:
            for result in results:
                # Добавляем столбец Language
                result_with_language = {
                    "language": result.get("language", "Unknown"),
                    "method": result["method"],
                    "text": result["text"],
                    "result": result["result"]
                }

                # Проверяем уникальность записи
                result_key = (result_with_language["language"], result_with_language["method"], result_with_language["text"])
                if result_key not in seen_results:
                    self.results.append(result_with_language)
                    seen_results.add(result_key)

    def _create_html_content(self):
        """
        Создает содержимое HTML-документа на основе результатов.
        """
        html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Combined Text Processing Report</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 20px;
        }
        h1 {
            color: #333;
        }
        table {
            width: 100%;
            border-collapse: collapse;
            margin-top: 20px;
        }
        th, td {
            border: 1px solid #ddd;
            padding: 8px;
            text-align: left;
        }
        th {
            background-color: #f4f4f4;
        }
        img {
            max-width: 100%;
            height: auto;
        }
    </style>
</head>
<body>
    <h1>Combined Text Processing Report</h1>
    <table>
        <tr>
            <th>Language</th>
            <th>Method</th>
            <th>Text</th>
            <th>Result</th>
        </tr>
"""
        for result in self.results:
            language = result["language"]
            method = result["method"]
            text = result["text"].replace("\n", "<br>")  # Обрабатываем переносы строк
            res = result["result"]

            if isinstance(res, dict) and "image" in res:  # Если результат содержит изображение
                image_path = res["image"]
                res_html = f'<img src="{os.path.basename(image_path)}" alt="{method}">'
            else:  # Иначе просто выводим текстовый результат
                res_html = str(res).replace("\n", "<br>")

            html_content += f"""
        <tr>
            <td>{language}</td>
            <td>{method}</td>
            <td>{text}</td>
            <td>{res_html}</td>
        </tr>
"""

        html_content += """
    </table>
</body>
</html>
"""
        return html_content

    def generate_report(self, output_filename="combined_report.html"):
        """
        Генерирует и сохраняет HTML-отчет в файл.

        :param output_filename: Имя файла для сохранения отчета.
        """
        html_content = self._create_html_content()

        # Определяем папку для сохранения изображений
        images_dir = "images"
        os.makedirs(images_dir, exist_ok=True)

        # Копируем все используемые изображения в папку images
        for result in self.results:
            if isinstance(result["result"], dict) and "image" in result["result"]:
                image_path = result["result"]["image"]
                new_image_path = os.path.join(images_dir, os.path.basename(image_path))
                os.replace(image_path, new_image_path)  # Перемещаем файл

        # Записываем HTML-контент в файл
        with open(output_filename, "w", encoding="utf-8") as f:
            f.write(html_content)

        print(f"Общий отчет успешно сгенерирован и сохранен в файле {output_filename}.")

In [31]:
if __name__ == "__main__":
    text_en = "Hello, world! This is an example of text processing in English."
    text_ru = "Привет, мир! Это пример обработки текста на русском языке."
    text_de = "Hallo Welt! Dies ist ein Beispiel für Textverarbeitung auf Deutsch."

    processor_en = TextProcessor(text_en, language='en')
    processor_ru = TextProcessor(text_ru, language='ru')
    processor_de = TextProcessor(text_de, language='de')

    # Выполнение всех методов для каждого процессора
    processor_en.execute_all_methods()
    processor_ru.execute_all_methods()
    processor_de.execute_all_methods()

    # Получение результатов для каждого процессора
    results_en = processor_en.get_results()
    results_ru = processor_ru.get_results()
    results_de = processor_de.get_results()

    # Объединение результатов
    all_results = [results_en, results_ru, results_de]

    # Создание и генерация общего HTML-отчета
    report_generator = HTMLReportGenerator(all_results)
    report_generator.generate_report("combined_text_processing_report.html")

spaCy model 'de_core_news_sm' is not installed. Attempting to install...
Model 'de_core_news_sm' has been successfully installed.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Stemming is not recommended for Russian. Using lemmatization instead.
Общий отчет успешно сгенерирован и сохранен в файле combined_text_processing_report.html.


#Векторизация

In [34]:
!pip install umap-learn fasttext langdetect allennlp

  Using cached allennlp-2.10.1-py3-none-any.whl.metadata (21 kB)
INFO: pip is looking at multiple versions of allennlp to determine which version is compatible with other requirements. This could take a while.
  Using cached allennlp-2.10.0-py3-none-any.whl.metadata (20 kB)
  Using cached allennlp-2.9.3-py3-none-any.whl.metadata (19 kB)
  Using cached allennlp-2.9.2-py3-none-any.whl.metadata (19 kB)
  Using cached allennlp-2.9.1-py3-none-any.whl.metadata (19 kB)
  Using cached allennlp-2.9.0-py3-none-any.whl.metadata (18 kB)
  Using cached allennlp-2.8.0-py3-none-any.whl.metadata (17 kB)
  Using cached allennlp-2.7.0-py3-none-any.whl.metadata (17 kB)
INFO: pip is still looking at multiple versions of allennlp to determine which version is compatible with other requirements. This could take a while.
  Using cached allennlp-2.6.0-py3-none-any.whl.metadata (17 kB)
  Using cached allennlp-2.5.0-py3-none-any.whl.metadata (17 kB)
  Using cached allennlp-2.4.0-py3-none-any.whl.metadata (17 kB

In [37]:
!pip install umap-learn

In [39]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder
from gensim.models import Word2Vec, Doc2Vec, FastText
from gensim.models.doc2vec import TaggedDocument
from gensim.models import KeyedVectors
from transformers import BertTokenizer, BertModel
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine
import torch
from gensim.models.doc2vec import TaggedDocument
from langdetect import detect, LangDetectException  # Добавьте этот импорт
import fasttext
import matplotlib.pyplot as plt
import os

class TextVectorizer:
    def __init__(self, texts, preprocess_with_textprocessor=True, language=None):
        """
        Инициализация класса с набором текстов.

        :param texts: Список строк (тексты для векторизации).
        :param preprocess_with_textprocessor: Флаг для использования TextProcessor для предобработки.
        :param language: Язык текстов (если None, язык будет определен автоматически).
        """
        self.texts = texts
        self.vocab = None
        self.vectors = None
        self.preprocess_with_textprocessor = preprocess_with_textprocessor
        self.language = language if language else self._detect_language()

        if self.preprocess_with_textprocessor:
            self.texts = self._preprocess_texts()

    def _detect_language(self):
        """Автоматическое определение языка первого текста."""
        try:
            return detect(self.texts[0]) if self.texts else "Unknown"
        except LangDetectException:
            return "Unknown"

    def _preprocess_texts(self):
        """Предобработка текстов с использованием TextProcessor."""
        processed_texts = []
        for text in self.texts:
            processor = TextProcessor(text, language=self.language)
            # Пример: выполняем стандартную токенизацию, удаляем стоп-слова и лемматизируем
            tokens = processor.remove_stopwords(processor.standard_tokenization())
            lemmas = processor.lemmatize(processor.clean_text())
            processed_texts.append(" ".join(lemmas))
        return processed_texts

    # 1. One-Hot Encoding
    def one_hot_encoding(self):
        """
        Векторизация текста с помощью One-Hot Encoding.
        """
        unique_words = set(word for text in self.texts for word in text.split())
        self.vocab = {word: i for i, word in enumerate(unique_words)}
        self.vectors = []

        for text in self.texts:
            vector = [0] * len(self.vocab)
            for word in text.split():
                if word in self.vocab:
                    vector[self.vocab[word]] = 1
            self.vectors.append(vector)

        return np.array(self.vectors)

    # 2. Bag of Words (BoW)
    def bag_of_words(self):
        """
        Векторизация текста с помощью Bag of Words.
        """
        vectorizer = CountVectorizer()
        self.vectors = vectorizer.fit_transform(self.texts).toarray()
        self.vocab = vectorizer.get_feature_names_out()
        return self.vectors

    # 3. TF-IDF
    def tf_idf(self):
        """
        Векторизация текста с помощью TF-IDF.
        """
        vectorizer = TfidfVectorizer()
        self.vectors = vectorizer.fit_transform(self.texts).toarray()
        self.vocab = vectorizer.get_feature_names_out()
        return self.vectors

    # 4. Word2Vec
    def word2vec(self, vector_size=100, window=5, min_count=1, workers=4):
        """
        Создание векторов слов с помощью Word2Vec.
        """
        tokenized_texts = [text.split() for text in self.texts]
        model = Word2Vec(sentences=tokenized_texts, vector_size=vector_size, window=window, min_count=min_count, workers=workers)
        self.vocab = list(model.wv.index_to_key)

        # Векторизация текстов путем усреднения векторов слов
        self.vectors = []
        for text in tokenized_texts:
            vec = np.mean([model.wv[word] for word in text if word in model.wv], axis=0)
            self.vectors.append(vec if np.any(vec) else np.zeros(vector_size))

        return np.array(self.vectors)

    # 5. Doc2Vec
    def doc2vec(self, vector_size=100, window=5, min_count=1, workers=4, epochs=20):
        """
        Создание векторов документов с помощью Doc2Vec.
        """
        tagged_data = [TaggedDocument(words=text.split(), tags=[str(i)]) for i, text in enumerate(self.texts)]
        model = Doc2Vec(vector_size=vector_size, window=window, min_count=min_count, workers=workers, epochs=epochs)
        model.build_vocab(tagged_data)
        model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

        self.vectors = [model.dv[str(i)] for i in range(len(self.texts))]
        return np.array(self.vectors)

    # 6. FastText
    def fasttext_embeddings(self, pretrained_model_path="cc.en.300.bin"):
        """
        Создание векторов слов с помощью FastText.
        """
        if not os.path.exists(pretrained_model_path):
            raise FileNotFoundError(f"Model file not found at {pretrained_model_path}")

        model = fasttext.load_model(pretrained_model_path)
        tokenized_texts = [text.split() for text in self.texts]
        self.vectors = []

        for text in tokenized_texts:
            vec = np.mean([model.get_word_vector(word) for word in text], axis=0)
            self.vectors.append(vec)

        return np.array(self.vectors)

    # 7. Contextual Embeddings (BERT)
    def contextual_embeddings(self, model_name="bert-base-uncased"):
        """
        Создание контекстуальных векторов с помощью BERT.
        """
        tokenizer = BertTokenizer.from_pretrained(model_name)
        model = BertModel.from_pretrained(model_name)

        inputs = tokenizer(self.texts, return_tensors="pt", padding=True, truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)

        # Берем векторы из последнего слоя
        self.vectors = outputs.last_hidden_state[:, 0, :].numpy()  # [CLS] токен
        return self.vectors

    # 8. Multilingual BERT
    def multilingual_bert_embeddings(self, model_name="bert-base-multilingual-cased"):
        """
        Создание контекстуальных векторов с помощью Multilingual BERT.
        """
        tokenizer = BertTokenizer.from_pretrained(model_name)
        model = BertModel.from_pretrained(model_name)

        inputs = tokenizer(self.texts, return_tensors="pt", padding=True, truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)

        self.vectors = outputs.last_hidden_state[:, 0, :].numpy()  # [CLS] токен
        return self.vectors

    # 9. ELMo
    def elmo_embeddings(self):
        """
        Создание контекстуальных векторов с помощью ELMo.
        """
        elmo = ElmoEmbedder()
        tokenized_texts = [text.split() for text in self.texts]
        self.vectors = []

        for tokens in tokenized_texts:
            embeddings = elmo.embed_sentence(tokens)
            mean_embedding = np.mean(embeddings, axis=0)  # Среднее по всем слоям
            self.vectors.append(np.mean(mean_embedding, axis=0))  # Среднее по словам

        return np.array(self.vectors)

    # 10. Universal Sentence Encoder
    def universal_sentence_encoder(self):
        """
        Создание векторов предложений с помощью Universal Sentence Encoder.
        """
        model = SentenceTransformer('all-MiniLM-L6-v2')
        self.vectors = model.encode(self.texts)
        return self.vectors

    # 11. Custom Embeddings
    def custom_embeddings(self, pretrained_model_path=None):
        """
        Создание пользовательских векторов с использованием предобученной модели.
        """
        if pretrained_model_path is None:
            raise ValueError("Path to pretrained model must be provided.")

        if not os.path.exists(pretrained_model_path):
            raise FileNotFoundError(f"Model file not found at {pretrained_model_path}")

        # Загрузка модели и создание векторов
        model = Word2Vec.load(pretrained_model_path)
        tokenized_texts = [text.split() for text in self.texts]
        self.vectors = []
        for text in tokenized_texts:
            vec = np.mean([model.wv[word] for word in text if word in model.wv], axis=0)
            self.vectors.append(vec if np.any(vec) else np.zeros(model.vector_size))

        return np.array(self.vectors)

    # 12. Сравнение векторов (косинусная близость)
    def compare_vectors(self, vector1, vector2):
        """
        Сравнение двух векторов с помощью косинусной близости.
        """
        return 1 - cosine(vector1, vector2)

    # 13. Визуализация векторов (PCA или UMAP)
    def visualize_vectors(self, method="pca", n_components=2):
        """
        Визуализация векторов с помощью PCA или UMAP.
        """
        if self.vectors is None:
            raise ValueError("Vectors are not computed yet.")

        if method == "pca":
            reducer = PCA(n_components=n_components)
        elif method == "umap":
            reducer = UMAP(n_components=n_components)
        else:
            raise ValueError("Unsupported visualization method. Use 'pca' or 'umap'.")

        reduced_vectors = reducer.fit_transform(self.vectors)

        plt.figure(figsize=(8, 6))
        if n_components == 2:
            plt.scatter(reduced_vectors[:, 0], reduced_vectors[:, 1])
            for i, txt in enumerate(self.texts):
                plt.annotate(txt[:10], (reduced_vectors[i, 0], reduced_vectors[i, 1]))
        elif n_components == 3:
            from mpl_toolkits.mplot3d import Axes3D
            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
            ax.scatter(reduced_vectors[:, 0], reduced_vectors[:, 1], reduced_vectors[:, 2])
        else:
            raise ValueError("n_components must be 2 or 3 for visualization.")

        plt.title(f"Vector Visualization using {method.upper()}")
        plt.show()

In [43]:
if __name__ == "__main__":
    # Исходные тексты
    texts = [
        "This is the first document.",
        "This document is the second document.",
        "And this is the third one.",
        "Is this the first document?"
    ]

    # Создаем экземпляр TextVectorizer с предобработкой через TextProcessor
    vectorizer = TextVectorizer(texts, preprocess_with_textprocessor=True)

    # Векторизация текста
    print("One-Hot Encoding:")
    vectorizer.one_hot_encoding()
    print(vectorizer.vectors)

    print("\nBag of Words:")
    vectorizer.bag_of_words()
    print(vectorizer.vectors)

    print("\nTF-IDF:")
    vectorizer.tf_idf()
    print(vectorizer.vectors)

    print("\nWord2Vec:")
    vectorizer.word2vec()
    print(vectorizer.vectors)

    print("\nDoc2Vec:")
    vectorizer.doc2vec()
    print(vectorizer.vectors)

    print("\nContextual Embeddings (BERT):")
    vectorizer.contextual_embeddings()
    print(vectorizer.vectors)

    print("\nMultilingual BERT:")
    vectorizer.multilingual_bert_embeddings()
    print(vectorizer.vectors)

    print("\nUniversal Sentence Encoder:")
    vectorizer.universal_sentence_encoder()
    print(vectorizer.vectors)

One-Hot Encoding:
[[0, 1, 1, 1, 1, 0, 0, 0, 1], [0, 1, 1, 0, 1, 1, 0, 0, 1], [1, 1, 1, 0, 0, 0, 1, 1, 1], [0, 1, 1, 1, 1, 0, 0, 0, 1]]

Bag of Words:
[[0 1 1 1 0 0 1 0 1]
 [0 1 2 0 0 1 1 0 1]
 [1 1 0 0 1 0 1 1 1]
 [0 1 1 1 0 0 1 0 1]]

TF-IDF:
[[0.         0.38408524 0.46979139 0.58028582 0.         0.
  0.38408524 0.         0.38408524]
 [0.         0.28108867 0.6876236  0.         0.         0.53864762
  0.28108867 0.         0.28108867]
 [0.51184851 0.26710379 0.         0.         0.51184851 0.
  0.26710379 0.51184851 0.26710379]
 [0.         0.38408524 0.46979139 0.58028582 0.         0.
  0.38408524 0.         0.38408524]]

Word2Vec:
[array([-4.8886943e-03,  3.5039980e-03, -7.7879167e-04,  1.8327751e-03,
        2.8311522e-03, -8.4003323e-04,  1.5665103e-03,  5.3411722e-03,
       -2.8435558e-03, -8.0334337e-04,  1.0816052e-03, -7.4949721e-04,
       -1.0159061e-03,  2.0650295e-03,  1.8931776e-03, -1.4124572e-03,
        4.1107098e-03,  3.7477990e-03, -4.5725266e-03, -2.6593199e-

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

[[-0.13980463  0.30051133  0.22007145 ... -0.35206625  0.32350102
   0.11706927]
 [-0.22906353  0.23592865  0.2669195  ... -0.5010979   0.59917057
   0.2358845 ]
 [-0.10073916  0.30093157 -0.0101437  ... -0.01799834  0.36438337
   0.37330565]
 [ 0.07388926  0.12602314 -0.07107982 ... -0.2659695   0.09015017
   0.21599169]]

Multilingual BERT:


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[[-0.05988198  0.09333178  0.03553385 ...  0.04377606  0.12508884
   0.03840595]
 [-0.07548568  0.08912562  0.02862243 ...  0.05355845  0.12087866
   0.02367232]
 [-0.03760171 -0.04521328 -0.01781185 ... -0.00820752  0.12598053
   0.01813569]
 [-0.07428253  0.09165493  0.0553926  ...  0.06479331  0.11478932
   0.04030686]]
